# Deep learning with tensorflow

## Convolutional network

## Create data
The data are generated by a python script in Blender :
- 900 point of views for each 54 cards.
- Each generated images as a resolution of 128 * 128.
- A total of 48 600 images.

## Create Data with blender

This step require [Blender](https://www.blender.org/download/) in order to generate the images
* Open TextureCards/cardgames.blend
* Run script (See screenshot below)

![Run blender script](https://raw.githubusercontent.com/Claudiooo/DeepLearningLearning/Group2/Images/blender_run.png)

### Blender script

```python
from os import listdir
from os import makedirs
from os.path import isfile, join, splitext, exists
from os.path import basename
import bpy

path = bpy.path.abspath("//")
onlyfiles = [f for f in listdir(path) if isfile(join(path, f)) and splitext(f)[1] == '.png']
for f in onlyfiles:
    img = img = bpy.data.images.load(path + f)
    tex = bpy.data.textures.new(f, 'IMAGE')
    tex.image = img
    
    bpy.data.materials['texture_card'].active_texture = tex
    dir = path + '..\\Data\\' + splitext(f)[0]
    if not exists(dir):
        makedirs(dir)
    bpy.context.scene.render.filepath = dir + "\\"
    bpy.ops.wm.console_toggle()
    print(bpy.context.scene.render.filepath)
    bpy.ops.render.render(animation=True)
```

## Import datasetlib

In [1]:
%run ../importlib.py

import datasetlib

importing Jupyter notebook from datasetlib.ipynb


## Define label value

In [2]:
label_tab = {
    '1_club' : 0, '1_diamond' : 1, '1_heart' : 2, '1_spade' : 3,
    '2_club' : 4, '2_diamond' : 5, '2_heart' : 6, '2_spade' : 7,
    '3_club' : 8, '3_diamond' : 9, '3_heart' : 10, '3_spade' : 11,
    '4_club' : 12, '4_diamond' : 13, '4_heart' : 14, '4_spade' : 15,
    '5_club' : 16, '5_diamond' : 17, '5_heart' : 18, '5_spade' : 19,
    '6_club' : 20, '6_diamond' : 21, '6_heart' : 22, '6_spade' : 23,
    '7_club' : 24, '7_diamond' : 25, '7_heart' : 26, '7_spade' : 27,
    '8_club' : 28, '8_diamond' : 29, '8_heart' : 30, '8_spade' : 31,
    '9_club' : 32, '9_diamond' : 33, '9_heart' : 34, '9_spade' : 35,
    '10_club' : 36, '10_diamond' : 37, '10_heart' : 38, '10_spade' : 39,
    'jack_club' : 40, 'jack_diamond' : 41, 'jack_heart' : 42, 'jack_spade' : 43,
    'queen_club' : 44, 'queen_diamond' : 45, 'queen_heart' : 46, 'queen_spade' : 47,
    'king_club' : 48, 'king_diamond' : 49, 'king_heart' : 50, 'king_spade' : 51,
    'joker_red' : 52, 'joker_black' : 53
}

## Define global variables

In [3]:
image_index_filename = 'image_cards.txt'
label_index_filename = 'label_cards.txt'
image_shuffle_index_filename = 'image_shuffle_cards.txt'
label_shuffle_index_filename = 'label_shuffle_cards.txt'
dataset_size = 0

## Generate index and label file

In [4]:
dataset_size = datasetlib.create_index_file(
    "Data/**/*.png",
    label_tab,
    image_index_filename,
    label_index_filename)

Create  image_cards.txt  :  1124236  bytes.
Create  label_cards.txt  :  131091  bytes.


In [5]:
dataset_size = datasetlib.create_shuffle_index_file(
    image_index_filename,
    label_index_filename,
    image_shuffle_index_filename,
    label_shuffle_index_filename)

Create  image_shuffle_cards.txt  :  1123998  bytes.
Create  label_shuffle_cards.txt  :  131098  bytes.


## Generate binary dataset

In [11]:
datasetlib.generate_chunck_dataset_thread(
    #souce file
    image_shuffle_index_filename,
    #number data per chunck
    10000,
    #number max of thread
    8,
    #number of chunck, -1 for automatic maximum chunck
    -1,
    #binary type 0 for image, 1 for label
    0)

Writing in image_shuffle_cards_0.ubyte has started...
Writing in image_shuffle_cards_1.ubyte has started...
Writing in image_shuffle_cards_2.ubyte has started...
Writing in image_shuffle_cards_3.ubyte has started...
Writing in image_shuffle_cards_4.ubyte has started...


VBox(children=(HTML(value=''), IntProgress(value=0, max=10000)))

VBox(children=(HTML(value=''), IntProgress(value=0, max=10000)))

VBox(children=(HTML(value=''), IntProgress(value=0, max=8600)))

VBox(children=(HTML(value=''), IntProgress(value=0, max=10000)))

VBox(children=(HTML(value=''), IntProgress(value=0, max=10000)))

Create file with thread n°4 : image_shuffle_cards_4.ubyte (422707216 bytes).
Create file with thread n°2 : image_shuffle_cards_2.ubyte (491520016 bytes).
Create file with thread n°3 : image_shuffle_cards_3.ubyte (491520016 bytes).
Create file with thread n°1 : image_shuffle_cards_1.ubyte (491520016 bytes).
Create file with thread n°0 : image_shuffle_cards_0.ubyte (491520016 bytes).


In [12]:
datasetlib.generate_dataset(image_shuffle_index_filename, dataset_size)

VBox(children=(HTML(value=''), IntProgress(value=0, max=48600)))

Create file : image_shuffle_cards.ubyte(2388787216 bytes).


## Configuration of Neural Network

In [7]:
import tensorflow as tf
import time

print(tf.__version__)

1.4.0


In [8]:
# Convolutional Layer 1.
filter_size1 = 22          # Convolution filters are 5 x 5 pixels.
num_filters1 = 72         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 22          # Convolution filters are 5 x 5 pixels.
num_filters2 = 162         # There are 36 of these filters.

# Fully-connected layer.
fc_size = 512             # Number of neurons in fully-connected layer.

## Data dimention

In [9]:
# We know that MNIST images are 28 pixels in each dimension.
img_size = 128

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = 3

# Number of classes, one class for each of 10 digits.
num_classes = 53

## Load data with tensorflow

In [10]:
from tensorflow.contrib.learn.python.learn.datasets import base
from tensorflow.python.framework import dtypes

def get_data_image_filename(filename, number, offset):
    file_dataset_image = open(filename,'r')
    dataset_image_tab = file_dataset_image.read().split(separator)
    dataset_image_tab.pop()
    tab = []
    size = offset + number
    if (size > len(dataset_image_tab)):
        size = len(dataset_image_tab)
    for i in range(offset, size):
#         image_string = tf.read_file(dataset_image_tab[i])
#         image_decoded = tf.image.decode_image(image_string, channels=3)
#         image_resized = tf.image.resize_images(tf.reshape(image_decoded, [128,128,3]), [128, 128])
#         tab.append(image_resized)
        tab.append(dataset_image_tab[i])
    return tab

def get_data_label(filename, number, offset):
    file_dataset_label = open(filename,'r')
    dataset_label_tab = file_dataset_label.read().split(separator)
    dataset_label_tab.pop()
    tab = []
    size = offset + number
    if (size > len(dataset_label_tab)):
        size = len(dataset_label_tab)
    for i in range(offset, size):
        tab.append(int(dataset_label_tab[i]))
    return tab

def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_image(image_string, channels=3)
    image_resized = tf.image.resize_images(tf.reshape(image_decoded, [128,128,3]), [128, 128])
    return image_resized, label

class Datasets(object):
    
    def __init__(self,
               images,
               labels):
        
        self._num_examples = images.shape[0]    
        self._images = images
        self._labels = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0
    
    @property
    def images(self):
        return self._images
    
    @property
    def num_examples(self):
        return self._num_examples

    @property
    def labels(self):
        return self._labels

    @property
    def epochs_completed(self):
        return self._epochs_completed
    
    def next_batch(self, batch_size):
        """Return the next `batch_size` examples from this data set."""
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self._num_examples:
          # Finished epoch
            self._epochs_completed += 1
          # Shuffle the data
            perm = np.arange(self._num_examples)
            np.random.shuffle(perm)
            self._images = self._images[perm]
            self._labels = self._labels[perm]
          # Start next epoch
            start = 0
            self._index_in_epoch = batch_size
            assert batch_size <= self._num_examples
        end = self._index_in_epoch
        return self._images[start:end], self._labels[start:end]

def load_data(number, offset=0):
    images = np.array(get_data_image_filename(dataset_image_shuffle.split[0] + '.ubyte', number, offset))
    labels = np.array(get_data_label(dataset_label_shuffle, number, offset))
    print("Images data : ", images)
    print("Label data : ", labels)
    train = Datasets(images, labels)
    return base.Datasets(train=train, validation=None, test=None)
    
    
    
# generate_ubyte_file(dataset_image_shuffle, 100)
dataset = load_data(100)

print(dataset)

NameError: name 'np' is not defined

In [ ]:
#dataset.test.cls = np.argmax(data.test.labels, axis=1)

## TensorFlow graph

In [ ]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

In [ ]:
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [ ]:
def new_conv_layer(input,              # The previous layer.
                   num_input_channels, # Num. channels in prev. layer.
                   filter_size,        # Width and height of each filter.
                   num_filters,        # Number of filters.
                   use_pooling=True):  # Use 2x2 max-pooling.

    # Shape of the filter-weights for the convolution.
    # This format is determined by the TensorFlow API.
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # Create new weights aka. filters with the given shape.
    weights = new_weights(shape=shape)

    # Create new biases, one for each filter.
    biases = new_biases(length=num_filters)

    # Create the TensorFlow operation for convolution.
    # Note the strides are set to 1 in all dimensions.
    # The first and last stride must always be 1,
    # because the first is for the image-number and
    # the last is for the input-channel.
    # But e.g. strides=[1, 2, 2, 1] would mean that the filter
    # is moved 2 pixels across the x- and y-axis of the image.
    # The padding is set to 'SAME' which means the input image
    # is padded with zeroes so the size of the output is the same.
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    # Add the biases to the results of the convolution.
    # A bias-value is added to each filter-channel.
    layer += biases

    # Use pooling to down-sample the image resolution?
    if use_pooling:
        # This is 2x2 max-pooling, which means that we
        # consider 2x2 windows and select the largest value
        # in each window. Then we move 2 pixels to the next window.
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    # Rectified Linear Unit (ReLU).
    # It calculates max(x, 0) for each input pixel x.
    # This adds some non-linearity to the formula and allows us
    # to learn more complicated functions.
    layer = tf.nn.relu(layer)

    # Note that ReLU is normally executed before the pooling,
    # but since relu(max_pool(x)) == max_pool(relu(x)) we can
    # save 75% of the relu-operations by max-pooling first.

    # We return both the resulting layer and the filter-weights
    # because we will plot the weights later.
    return layer, weights

In [ ]:
def flatten_layer(layer):
    # Get the shape of the input layer.
    layer_shape = layer.get_shape()

    # The shape of the input layer is assumed to be:
    # layer_shape == [num_images, img_height, img_width, num_channels]

    # The number of features is: img_height * img_width * num_channels
    # We can use a function from TensorFlow to calculate this.
    num_features = layer_shape[1:4].num_elements()
    
    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    layer_flat = tf.reshape(layer, [-1, num_features])

    # The shape of the flattened layer is now:
    # [num_images, img_height * img_width * num_channels]

    # Return both the flattened layer and the number of features.
    return layer_flat, num_features

In [ ]:

def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True): # Use Rectified Linear Unit (ReLU)?

    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')

In [ ]:
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

In [ ]:
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

In [ ]:
y_true_cls = tf.argmax(y_true, dimension=1)

In [ ]:
layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   use_pooling=True)

In [ ]:
layer_conv1

In [ ]:

layer_conv2, weights_conv2 = \
    new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=True)

In [ ]:
layer_conv2


In [ ]:
layer_flat, num_features = flatten_layer(layer_conv2)


In [ ]:
layer_flat

In [ ]:
num_features

In [ ]:
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         use_relu=True)

In [ ]:
layer_fc1

In [ ]:
layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         use_relu=False)

In [ ]:
layer_fc2

In [ ]:
y_pred = tf.nn.softmax(layer_fc2)

In [ ]:
y_pred_cls = tf.argmax(y_pred, dimension=1)

In [ ]:

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                        labels=y_true)

In [ ]:

cost = tf.reduce_mean(cross_entropy)

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)


In [ ]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)


In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [ ]:

session = tf.Session()

In [ ]:
session.run(tf.global_variables_initializer())

In [ ]:
train_batch_size = 64

In [ ]:
# Counter for total number of iterations performed so far.
total_iterations = 0

def optimize(num_iterations):
    # Ensure we update the global variable rather than a local copy.
    global total_iterations

    # Start-time used for printing time-usage below.
    start_time = time.time()

    for i in range(total_iterations,
                   total_iterations + num_iterations):

        # Get a batch of training examples.
        # x_batch now holds a batch of images and
        # y_true_batch are the true labels for those images.
        print(train_batch_size)
        x_batch, y_true_batch = dataset.train.next_batch(train_batch_size)

        # Put the batch into a dict with the proper names
        # for placeholder variables in the TensorFlow graph.
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}

        # Run the optimizer using this batch of training data.
        # TensorFlow assigns the variables in feed_dict_train
        # to the placeholder variables and then runs the optimizer.
        print(feed_dict_train)
        session.run(optimizer, feed_dict=feed_dict_train)

        # Print status every 100 iterations.
        if i % 100 == 0:
            # Calculate the accuracy on the training-set.
            acc = session.run(accuracy, feed_dict=feed_dict_train)

            # Message for printing.
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"

            # Print it.
            print(msg.format(i + 1, acc))

    # Update the total number of iterations performed.
    total_iterations += num_iterations

    # Ending time.
    end_time = time.time()

    # Difference between start and end-times.
    time_dif = end_time - start_time

    # Print the time-usage.
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

In [ ]:
# Split the test-set into smaller batches of this size.
test_batch_size = 256

def print_test_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):

    # Number of images in the test-set.
    num_test = len(dataset.test.images)

    # Allocate an array for the predicted classes which
    # will be calculated in batches and filled into this array.
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    # Now calculate the predicted classes for the batches.
    # We will just iterate through all the batches.
    # There might be a more clever and Pythonic way of doing this.

    # The starting index for the next batch is denoted i.
    i = 0

    while i < num_test:
        # The ending index for the next batch is denoted j.
        j = min(i + test_batch_size, num_test)

        # Get the images from the test-set between index i and j.
        images = dataset.test.images[i:j, :]

        # Get the associated labels.
        labels = dataset.test.labels[i:j, :]

        # Create a feed-dict with these images and labels.
        feed_dict = {x: images,
                     y_true: labels}

        # Calculate the predicted class using TensorFlow.
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        # Set the start-index for the next batch to the
        # end-index of the current batch.
        i = j

    # Convenience variable for the true class-numbers of the test-set.
    cls_true = dataset.test.cls

    # Create a boolean array whether each image is correctly classified.
    correct = (cls_true == cls_pred)

    # Calculate the number of correctly classified images.
    # When summing a boolean array, False means 0 and True means 1.
    correct_sum = correct.sum()

    # Classification accuracy is the number of correctly classified
    # images divided by the total number of images in the test-set.
    acc = float(correct_sum) / num_test

    # Print the accuracy.
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))

    # Plot some examples of mis-classifications, if desired.
    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    # Plot the confusion matrix, if desired.
    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)

In [ ]:
#print_test_accuracy()

optimize(num_iterations=1)